In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Por cuestiones de tamanio, no se incluye el dataset debido a que este pesa mas de 1GB, para ejecutar, dicho dataset debe estar en la
#carpeta data

# Configuración de visualización
plt.style.use('seaborn')
sns.set_palette("husl")

# 1. Carga y Exploración de Datos
print("Cargando muestra del dataset...")

# Calculamos el tamaño de la muestra (usando 100,000 registros o 5% del dataset, el que sea mayor)
sample_rows = 100_000  # Podemos ajustar este número según necesidad

# Primero leemos el número total de filas sin cargar el dataset
total_rows = sum(1 for _ in open('../../../data/vehicles.csv')) - 1 
sampling_rate = sample_rows / total_rows

# Cargamos una muestra aleatoria del dataset
selected_columns = ['price', 'year', 'manufacturer', 'model', 'condition', 
                   'odometer', 'fuel', 'transmission', 'drive', 'state']

df = pd.read_csv('vehicles.csv', 
                 usecols=selected_columns,
                 skiprows=lambda x: x>0 and np.random.random() > sampling_rate)

print(f"Muestra cargada: {len(df)} registros de {total_rows} ({(len(df)/total_rows*100):.1f}%)")

# Mostrar información básica del dataset
print("\nDimensiones de la muestra:", df.shape)
print("\nPrimeras 5 filas:")
print(df.head())
print("\nEstadísticas descriptivas:")
print(df.describe())
print("\nValores faltantes por columna:")
print(df.isnull().sum())
print("\nDuplicados:", df.duplicated().sum())

# 2. Limpieza y Preprocesamiento
print("\nIniciando limpieza y preprocesamiento...")

# Eliminar duplicados
df.drop_duplicates(inplace=True)

# Convertir columnas numéricas y manejar valores faltantes
numeric_columns = ['price', 'year', 'odometer']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col].fillna(df[col].median(), inplace=True)

# Manejar valores faltantes en columnas categóricas
categorical_columns = ['manufacturer', 'model', 'condition', 'fuel', 
                      'transmission', 'drive', 'state']
for col in categorical_columns:
    df[col].fillna('unknown', inplace=True)

# Eliminar outliers extremos de precio
Q1 = df['price'].quantile(0.01)
Q3 = df['price'].quantile(0.99)
IQR = Q3 - Q1
df = df[~((df['price'] < (Q1 - 1.5 * IQR)) | (df['price'] > (Q3 + 1.5 * IQR)))]

# Codificar variables categóricas
le = LabelEncoder()
for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

# Escalar características numéricas
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 3. Exploración de Datos
print("\nCreando visualizaciones...")

fig = plt.figure(figsize=(15, 12))

# Distribución de precios
plt.subplot(2, 2, 1)
sns.histplot(data=df, x='price', kde=True)
plt.title('Distribución de Precios')

# Relación precio vs año
plt.subplot(2, 2, 2)
sns.scatterplot(data=df, x='year', y='price', alpha=0.5)
plt.title('Precio vs Año')

# Relación precio vs odómetro
plt.subplot(2, 2, 3)
sns.scatterplot(data=df, x='odometer', y='price', alpha=0.5)
plt.title('Precio vs Odómetro')

# Precios por estado
plt.subplot(2, 2, 4)
plt.xticks(rotation=45)
sns.boxplot(data=df, x='state', y='price')
plt.title('Precios por Estado')

plt.tight_layout()
plt.show()

# Matriz de correlación para variables numéricas
plt.figure(figsize=(10, 8))
correlation_matrix = df[numeric_columns].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación')
plt.show()

# 4. Modelado y Evaluación
print("\nPreparando datos para modelado...")

# Preparar datos
X = df.drop('price', axis=1)
y = df['price']

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nEntrenando modelos...")

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)

print("\nResultados Linear Regression:")
lr_mse = mean_squared_error(y_test, lr_pred)
lr_rmse = np.sqrt(lr_mse)
lr_r2 = r2_score(y_test, lr_pred)
print(f"MSE: {lr_mse:.2f}")
print(f"RMSE: {lr_rmse:.2f}")
print(f"R2: {lr_r2:.2f}")

# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

print("\nResultados Random Forest:")
rf_mse = mean_squared_error(y_test, rf_pred)
rf_rmse = np.sqrt(rf_mse)
rf_r2 = r2_score(y_test, rf_pred)
print(f"MSE: {rf_mse:.2f}")
print(f"RMSE: {rf_rmse:.2f}")
print(f"R2: {rf_r2:.2f}")

# 5. Optimización del Modelo
print("\nOptimizando Random Forest...")

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("\nMejores parámetros:", grid_search.best_params_)
print("Mejor score:", -grid_search.best_score_)

# Entrenar el modelo final con los mejores parámetros
final_model = RandomForestRegressor(**grid_search.best_params_, random_state=42, n_jobs=-1)
final_model.fit(X_train, y_train)
final_pred = final_model.predict(X_test)

print("\nResultados del modelo optimizado:")
final_mse = mean_squared_error(y_test, final_pred)
final_rmse = np.sqrt(final_mse)
final_r2 = r2_score(y_test, final_pred)
print(f"MSE: {final_mse:.2f}")
print(f"RMSE: {final_rmse:.2f}")
print(f"R2: {final_r2:.2f}")

# Importancia de características
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': final_model.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Importancia de Características')
plt.show()